In [3]:
import pandas as pd
import numpy as np

In [4]:
path="C:/Users/manas/my-project/cleaned_legal_metadata.csv"
df=pd.read_csv(path)
print(df.head())

         id                                               name  \
0  12532561  Jessica OLSEN, on behalf of herself and all ot...   
1  12532560  AMERICAN CENTER FOR LAW AND JUSTICE, Plaintiff...   
2  12532562  UNITED STATES of America, Plaintiff, v. Steven...   
3  12532563  Kaori STEARNEY, et al., Plaintiffs, v. UNITED ...   
4  12532564  IN RE: GERMAN AUTOMOTIVE MANUFACTURERS ANTITRU...   

                                   name_abbreviation decision_date  \
0                              Olsen v. Nelnet, Inc.    2019-05-21   
1  Am. Ctr. for Law & Justice v. U.S. Dep't of Ju...    2019-06-30   
2                             United States v. Emery    2019-06-21   
3                          Stearney v. United States    2019-05-16   
4          In re German Auto. Mfrs. Antitrust Litig.    2019-06-17   

                    docket_number court/name_abbreviation  
0                    4:18-CV-3081                 D. Neb.  
1  Civil Action No. 16-2188 (TJK)                  D.D.C.  
2 

In [5]:
print(df.tail())

          id                                               name  \
84  12532643  WORLD WATER WORKS HOLDINGS, INC., Plaintiff, v...   
85  12532644  Caitlin BERNARD M.D., Plaintiff, v. INDIVIDUAL...   
86  12532645  REXING QUALITY EGGS, Plaintiff, v. REMBRANDT E...   
87  12532646  Osha JOSEPH, Plaintiff, v. Sgt. Bobby DONAHUE,...   
88  12532647  ZURICH AMERICAN INSURANCE COMPANY, Plaintiff, ...   

                                    name_abbreviation decision_date  \
84  World Water Works Holdings, Inc. v. Cont'l Cas...    2019-06-24   
85  Bernard v. Individual Members of the Ind. Med....    2019-06-28   
86                    Eggs v. Rembrandt Enters., Inc.    2019-05-29   
87                                  Joseph v. Donahue    2019-05-28   
88          Zurich Am. Ins. Co. v. Ins. Co. of N. Am.    2019-05-21   

                docket_number court/name_abbreviation  
84             No. 17 CV 5237               N.D. Ill.  
85  No. 1:19-cv-01660-SEB-DML               S.D. Ind.  
86

In [6]:
print(df.columns)
print(df.isnull().sum())

Index(['id', 'name', 'name_abbreviation', 'decision_date', 'docket_number',
       'court/name_abbreviation'],
      dtype='object')
id                         0
name                       0
name_abbreviation          0
decision_date              0
docket_number              0
court/name_abbreviation    0
dtype: int64


In [9]:
print(df.shape)

(89, 6)


In [11]:
print(df.dtypes)

id                          int64
name                       object
name_abbreviation          object
decision_date              object
docket_number              object
court/name_abbreviation    object
dtype: object
